In [1]:
from time import perf_counter

from qiskit.circuit import QuantumCircuit
from qiskit_aer.primitives import Sampler

from qtpu.compiler.compiler import compile_reach_size
from qtpu.evaluate import evaluate_sampler

from benchmark.ckt import run_qtpu, cut_ckt, run_ckt
from benchmark.ansatz import generate_ansatz
from benchmark.util import append_to_csv, DummySampler

In [2]:
# QPU_SIZES = [5, ]
XS, S, M, L, XL = 15, 30, 70, 100, 200

In [3]:
benchmarks = [
    {"name": "linear", "num_qubits": size, "reps": 2}
    for size in range(30, 101, 10)
]

In [6]:
def compile_and_run_qtpu(circuit, qpu_size):
    start = perf_counter()
    circuit = circuit.copy()
    circuit.measure_all()
    cut_circ = compile_reach_size(circuit, qpu_size, show_progress_bar=True)
    # print(cut_circ)
    compile_time = perf_counter() - start
    # evaluate_sampler(Sampler(), 10000)
    res, data = run_qtpu(cut_circ, 0.0, evaluate_sampler(DummySampler()))
    print(res)

    return {"qtpu_compile": compile_time, **data}


def compile_and_run_ckt(circuit, qpu_size):
    start = perf_counter()
    cut_circ = cut_ckt(circuit, qpu_size)
    compile_time = perf_counter() - start

    res, data = run_ckt(cut_circ, DummySampler(), 1000)
    print(res)

    return {"ckt_compile": compile_time, **data}

In [7]:
QPU_SIZE = 15
CSV = "runtime_results2.csv"

for bench in benchmarks:
    circuit = generate_ansatz(**bench)
    # print(run_cutensor(circuit))
    qtpu_res = compile_and_run_qtpu(circuit, QPU_SIZE)
    print(qtpu_res)
    ckt_res = compile_and_run_ckt(circuit, QPU_SIZE)
    print(ckt_res)
    append_to_csv(CSV, {**bench, **qtpu_res, **ckt_res})

  0%|          | 0/100 [00:00<?, ?it/s]

[FrozenTrial(number=0, state=1, values=[46008.0, 0.992329186985131], datetime_start=datetime.datetime(2024, 8, 12, 16, 47, 42, 14818), datetime_complete=datetime.datetime(2024, 8, 12, 16, 47, 42, 30086), params={'compress': 'qubits', 'choose_leaf': 'qubits', 'random_strength': 1.0025061066506056, 'weight_edges': 'log', 'imbalance': 0.17081292589158625, 'imbalance_decay': -3.725388080569073, 'parts': 11, 'parts_decay': 0.974101580890935, 'mode': 'recursive', 'objective': 'km1', 'fix_output_nodes': ''}, user_attrs={'ir': <qtpu.compiler.compress.CompressedIR object at 0x7fc3eeb2e490>, 'tree': <ContractionTree(N=30, branches=8, complete=False)>}, system_attrs={'nsga2:generation': 0}, intermediate_values={}, distributions={'compress': CategoricalDistribution(choices=('qubits', '2q', 'none')), 'choose_leaf': CategoricalDistribution(choices=('qubits',)), 'random_strength': FloatDistribution(high=10.0, log=False, low=0.01, step=None), 'weight_edges': CategoricalDistribution(choices=('const', '

  0%|          | 0/100 [00:00<?, ?it/s]

[FrozenTrial(number=3, state=1, values=[13203.0, 0.9890597347560979], datetime_start=datetime.datetime(2024, 8, 12, 16, 48, 11, 849486), datetime_complete=datetime.datetime(2024, 8, 12, 16, 48, 11, 863248), params={'compress': 'qubits', 'choose_leaf': 'qubits', 'random_strength': 1.1580657782342145, 'weight_edges': 'const', 'imbalance': 0.7573435320344877, 'imbalance_decay': -1.4572119159488892, 'parts': 6, 'parts_decay': 0.9762256772264701, 'mode': 'direct', 'objective': 'km1', 'fix_output_nodes': 'auto'}, user_attrs={'ir': <qtpu.compiler.compress.CompressedIR object at 0x7fc3ed469e50>, 'tree': <ContractionTree(N=40, branches=3, complete=False)>}, system_attrs={'nsga2:generation': 0}, intermediate_values={}, distributions={'compress': CategoricalDistribution(choices=('qubits', '2q', 'none')), 'choose_leaf': CategoricalDistribution(choices=('qubits',)), 'random_strength': FloatDistribution(high=10.0, log=False, low=0.01, step=None), 'weight_edges': CategoricalDistribution(choices=('con

  0%|          | 0/100 [00:00<?, ?it/s]

...

[FrozenTrial(number=5, state=1, values=[39447.0, 0.9890597347560979], datetime_start=datetime.datetime(2024, 8, 12, 16, 54, 40, 289695), datetime_complete=datetime.datetime(2024, 8, 12, 16, 54, 40, 314716), params={'compress': 'qubits', 'choose_leaf': 'qubits', 'random_strength': 9.832464091454781, 'weight_edges': 'const', 'imbalance': 0.3222724139317601, 'imbalance_decay': -0.3401894400493264, 'parts': 8, 'parts_decay': 0.6194958856928221, 'mode': 'recursive', 'objective': 'km1', 'fix_output_nodes': ''}, user_attrs={'ir': <qtpu.compiler.compress.CompressedIR object at 0x7fc3d0f7b0d0>, 'tree': <ContractionTree(N=80, branches=7, complete=False)>}, system_attrs={'nsga2:generation': 0}, intermediate_values={}, distributions={'compress': CategoricalDistribution(choices=('qubits', '2q', 'none')), 'choose_leaf': CategoricalDistribution(choices=('qubits',)), 'random_strength': FloatDistribution(high=10.0, log=False, low=0.01, step=None), 'weight_edges': CategoricalDistribution(choices=('const

  0%|          | 0/100 [00:00<?, ?it/s]

[FrozenTrial(number=42, state=1, values=[59130.0, 0.9901483535267246], datetime_start=datetime.datetime(2024, 8, 12, 16, 57, 19, 314716), datetime_complete=datetime.datetime(2024, 8, 12, 16, 57, 19, 346342), params={'compress': 'qubits', 'choose_leaf': 'qubits', 'random_strength': 4.586287478079866, 'weight_edges': 'const', 'imbalance': 0.5481938476303521, 'imbalance_decay': 2.1462634385873827, 'parts': 11, 'parts_decay': 0.9308936229474869, 'mode': 'direct', 'objective': 'cut', 'fix_output_nodes': ''}, user_attrs={'ir': <qtpu.compiler.compress.CompressedIR object at 0x7fc3cf2c61d0>, 'tree': <ContractionTree(N=90, branches=10, complete=False)>}, system_attrs={'nsga2:generation': 0}, intermediate_values={}, distributions={'compress': CategoricalDistribution(choices=('qubits', '2q', 'none')), 'choose_leaf': CategoricalDistribution(choices=('qubits',)), 'random_strength': FloatDistribution(high=10.0, log=False, low=0.01, step=None), 'weight_edges': CategoricalDistribution(choices=('const'

  0%|          | 0/100 [00:00<?, ?it/s]

[FrozenTrial(number=1, state=1, values=[32886.0, 0.983634567837717], datetime_start=datetime.datetime(2024, 8, 12, 16, 59, 28, 820338), datetime_complete=datetime.datetime(2024, 8, 12, 16, 59, 28, 851570), params={'compress': 'qubits', 'choose_leaf': 'qubits', 'random_strength': 8.898428900161589, 'weight_edges': 'const', 'imbalance': 0.7863367037635384, 'imbalance_decay': 1.5463955906651012, 'parts': 7, 'parts_decay': 0.7173001899288962, 'mode': 'direct', 'objective': 'km1', 'fix_output_nodes': ''}, user_attrs={'ir': <qtpu.compiler.compress.CompressedIR object at 0x7fc3e960c510>, 'tree': <ContractionTree(N=100, branches=6, complete=False)>}, system_attrs={'nsga2:generation': 0}, intermediate_values={}, distributions={'compress': CategoricalDistribution(choices=('qubits', '2q', 'none')), 'choose_leaf': CategoricalDistribution(choices=('qubits',)), 'random_strength': FloatDistribution(high=10.0, log=False, low=0.01, step=None), 'weight_edges': CategoricalDistribution(choices=('const', '